In [276]:
import numpy as np
import pandas as pd
import importlib

In [277]:
import common

# モジュールの再読み込み
importlib.reload(common)

_common = common.Common()
_common.BASE_PATH

PosixPath('/Users/iwasakitakahiro/github')

In [278]:
train_df = pd.read_csv('../input/学習用データ/train.csv', usecols=_common.TRAIN_FEATURE_COLS)
test_df = pd.read_csv('../input/評価用データ/test.csv', usecols=_common.TEST_FEATURE_COLS)

In [279]:
train_df['time'] = pd.to_datetime(train_df['time'], utc=True)
test_df['time'] = pd.to_datetime(test_df['time'], utc=True)

## 欠損値補完

In [280]:
# 前後の値による線形補完
missing_cols_train = train_df.columns[train_df.isnull().any()].tolist()
missing_cols_test = test_df.columns[test_df.isnull().any()].tolist()

train_df[missing_cols_train] = train_df[missing_cols_train].interpolate(method='linear')
test_df[missing_cols_test] = test_df[missing_cols_test].interpolate(method='linear')

## 特徴量エンジニアリング

### 発電・供給データ

#### 発電量合計

In [281]:
# "generation" を含む列名だけを抽出
gene_cols = [col for col in train_df.columns if 'generation' in col]

# 合計を新しい列として追加
train_df['gene_sum'] = train_df[gene_cols].sum(axis=1)
test_df['gene_sum'] = test_df[gene_cols].sum(axis=1)

#### 再生可能エネルギー比率

In [282]:
# train_df['sus_amount'] = train_df[_common.SUS_GENE_COLS].sum(axis=1)
# test_df['sus_amount'] = test_df[_common.SUS_GENE_COLS].sum(axis=1)

#### 火力発電比率

In [283]:
# fossil_cols = [col for col in train_df.columns if 'generation_fossil' in col]
# train_df['fossil_amount'] = train_df[fossil_cols].sum(axis=1)
# test_df['fossil_amount'] = test_df[fossil_cols].sum(axis=1)

#### 発電量と供給量の比

In [284]:
train_df['gene_load_ratio'] = train_df['gene_sum'] / train_df['total_load_actual']
test_df['gene_load_ratio'] = test_df['gene_sum'] / test_df['total_load_actual']

#### 発電コストの算出

In [285]:
# # 各発電方式のコスト辞書（EUR/MWh）
# cost_dict = {
#     'biomass': 100,
#     'brown_coal/lignite': 60,
#     'gas': 70,
#     'coal': 80,
#     'oil': 150,
#     'pump_hydro': 0,
#     'runofriver_hydro': 45,
#     'dam_hydro': 60,
#     'nuclear': 50,
#     'other': 80,
#     'other_renewables': 85,
#     'solar': 35,
#     'waste': 100,
#     'wind_onshore': 45
# }

# # 各発電量列 × コスト → 発電コスト列を作成
# train_df['gene_cost'] = (
#     train_df['generation_biomass'] * cost_dict['biomass'] +
#     train_df['generation_fossil_brown_coal/lignite'] * cost_dict['brown_coal/lignite'] +
#     train_df['generation_fossil_gas'] * cost_dict['gas'] +
#     train_df['generation_fossil_hard_coal'] * cost_dict['coal'] +
#     train_df['generation_fossil_oil'] * cost_dict['oil'] +
#     train_df['generation_hydro_pumped_storage_consumption'] * cost_dict['pump_hydro'] +
#     train_df['generation_hydro_run_of_river_and_poundage'] * cost_dict['runofriver_hydro'] +
#     train_df['generation_hydro_water_reservoir'] * cost_dict['dam_hydro'] +
#     train_df['generation_nuclear'] * cost_dict['nuclear'] +
#     train_df['generation_other'] * cost_dict['other'] +
#     train_df['generation_other_renewable'] * cost_dict['other_renewables'] +
#     train_df['generation_solar'] * cost_dict['solar'] +
#     train_df['generation_waste'] * cost_dict['waste'] +
#     train_df['generation_wind_onshore'] * cost_dict['wind_onshore']
# )

# test_df['gene_cost'] = (
#     test_df['generation_biomass'] * cost_dict['biomass'] +
#     test_df['generation_fossil_brown_coal/lignite'] * cost_dict['brown_coal/lignite'] +
#     test_df['generation_fossil_gas'] * cost_dict['gas'] +
#     test_df['generation_fossil_hard_coal'] * cost_dict['coal'] +
#     test_df['generation_fossil_oil'] * cost_dict['oil'] +
#     test_df['generation_hydro_pumped_storage_consumption'] * cost_dict['pump_hydro'] +
#     test_df['generation_hydro_run_of_river_and_poundage'] * cost_dict['runofriver_hydro'] +
#     test_df['generation_hydro_water_reservoir'] * cost_dict['dam_hydro'] +
#     test_df['generation_nuclear'] * cost_dict['nuclear'] +
#     test_df['generation_other'] * cost_dict['other'] +
#     test_df['generation_other_renewable'] * cost_dict['other_renewables'] +
#     test_df['generation_solar'] * cost_dict['solar'] +
#     test_df['generation_waste'] * cost_dict['waste'] +
#     test_df['generation_wind_onshore'] * cost_dict['wind_onshore']
# )

#### 残余需要、残余比率

In [286]:
# 残余需要 = 総需要量 - 再エネ発電量
train_df['residual_demand'] = train_df['total_load_actual'] - train_df[_common.SUS_GENE_COLS].sum(axis=1)
test_df['residual_demand'] = test_df['total_load_actual'] - test_df[_common.SUS_GENE_COLS].sum(axis=1)

# # 残余比率 = 残余需要 ÷ 総需要量
# train_df['residual_demand_ratio'] = train_df['residual_demand'] / train_df['total_load_actual']
# test_df['residual_demand_ratio'] = test_df['residual_demand'] / test_df['total_load_actual']

#### メリットオーダー（電力市場で電力を供給する順番を、発電コストが安い順に並べたもの）を用いた特徴量

In [287]:
# # 加重平均発電コスト（€/MWh）
# train_df['w_gene_cost'] = train_df['gene_cost'] / train_df['gene_sum']
# test_df['w_gene_cost'] = test_df['gene_cost'] / test_df['gene_sum']

In [288]:
# 高コスト発電比率
train_df['high_cost_ratio'] = (train_df['generation_fossil_gas'] + train_df['generation_fossil_gas'] + train_df['generation_fossil_oil']) / train_df['gene_sum']
test_df['high_cost_ratio'] = (test_df['generation_fossil_gas'] + test_df['generation_fossil_gas'] + test_df['generation_fossil_oil']) / test_df['gene_sum']

In [289]:
# train_df = train_df.drop(columns=gene_cols)
# test_df = test_df.drop(columns=gene_cols)

#### 需給逼迫フラグ
残余需要が過去平均と比べて高水準であればフラグを立てる

In [290]:
q3_residual = train_df['residual_demand'].quantile(0.75)

train_df['tight_supply_flag'] = (train_df['residual_demand'] > q3_residual).astype(int)
test_df['tight_supply_flag']  = (test_df['residual_demand']  > q3_residual).astype(int)

### 時系列特徴量

#### 時間の三角関数の変換

In [291]:
train_df['hour_sin'] = np.sin(2 * np.pi * train_df[_common.UNIQUE_KEY_COLS].dt.hour / 24)
train_df['hour_cos'] = np.cos(2 * np.pi * train_df[_common.UNIQUE_KEY_COLS].dt.hour / 24)

test_df['hour_sin'] = np.sin(2 * np.pi * test_df[_common.UNIQUE_KEY_COLS].dt.hour / 24)
test_df['hour_cos'] = np.cos(2 * np.pi * test_df[_common.UNIQUE_KEY_COLS].dt.hour / 24)

In [292]:
train_df['weekday_sin'] = np.sin(2 * np.pi * train_df[_common.UNIQUE_KEY_COLS].dt.weekday / 7)
train_df['weekday_cos'] = np.cos(2 * np.pi * train_df[_common.UNIQUE_KEY_COLS].dt.weekday / 7)

test_df['weekday_sin'] = np.sin(2 * np.pi * test_df[_common.UNIQUE_KEY_COLS].dt.weekday / 7)
test_df['weekday_cos'] = np.cos(2 * np.pi * test_df[_common.UNIQUE_KEY_COLS].dt.weekday / 7)

In [293]:
train_df['month_sin'] = np.sin(2 * np.pi * train_df[_common.UNIQUE_KEY_COLS].dt.month / 12)
train_df['month_cos'] = np.cos(2 * np.pi * train_df[_common.UNIQUE_KEY_COLS].dt.month / 12)

test_df['month_sin'] = np.sin(2 * np.pi * test_df[_common.UNIQUE_KEY_COLS].dt.month / 12)
test_df['month_cos'] = np.cos(2 * np.pi * test_df[_common.UNIQUE_KEY_COLS].dt.month / 12)

In [294]:
train_df['quarter_sin'] = np.sin(2 * np.pi * train_df[_common.UNIQUE_KEY_COLS].dt.quarter / 4)
train_df['quarter_cos'] = np.cos(2 * np.pi * train_df[_common.UNIQUE_KEY_COLS].dt.quarter / 4)

test_df['quarter_sin'] = np.sin(2 * np.pi * test_df[_common.UNIQUE_KEY_COLS].dt.quarter / 4)
test_df['quarter_cos'] = np.cos(2 * np.pi * test_df[_common.UNIQUE_KEY_COLS].dt.quarter / 4)

In [295]:
# TODO: クオーターも作る、dayofyearインデックスの効果を調べる

In [296]:
train_df['hour'] = np.arctan2(train_df['hour_sin'], train_df['hour_cos'])
train_df['weekday'] = np.arctan2(train_df['weekday_cos'], train_df['weekday_sin'])
train_df['month'] = np.arctan2(train_df['month_cos'], train_df['month_sin'])
train_df['quarter'] = np.arctan2(train_df['quarter_cos'], train_df['quarter_sin'])
test_df['hour'] = np.arctan2(test_df['hour_cos'], test_df['hour_sin'])
test_df['weekday'] = np.arctan2(test_df['weekday_cos'], test_df['weekday_sin'])
test_df['month'] = np.arctan2(test_df['month_cos'], test_df['month_sin'])
test_df['quarter'] = np.arctan2(test_df['quarter_cos'], test_df['quarter_sin'])

train_df = train_df.drop(columns=['hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos', 'month_sin', 'month_cos', 'quarter_sin', 'quarter_cos'])
test_df = test_df.drop(columns=['hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos', 'month_sin', 'month_cos', 'quarter_sin', 'quarter_cos'])

In [297]:
train_df['dayofyear'] = train_df[_common.UNIQUE_KEY_COLS].dt.dayofyear
test_df['dayofyear']  = test_df[_common.UNIQUE_KEY_COLS].dt.dayofyear

#### 発電量・供給量の時系列特徴量（ラグ、ローリング、エクスパンディング）

In [298]:
# まず識別用フラグ追加（どちらがtrainかtestか後でわかるように）
train_df['is_train'] = True
test_df['is_train'] = False

# train, testを結合（indexを一意にしたい場合はreset_index(drop=True)も可）
all_df = pd.concat([train_df, test_df], axis=0, sort=False)

# ---- ここでラグ・ローリング特徴量を一括計算 ----
cols = ['gene_sum', 'total_load_actual']
lag_hours       = [1, 24, 48, 72]
rolling_windows = [168, 336, 672]     # 7, 14, 28 日
scale_cols      = cols                # z-score を作る対象

for col in cols:
    # # --- ラグ ---
    # for lag in lag_hours:
    #     all_df[f'{col}_lag{lag}'] = all_df[col].shift(lag)

    # # --- ローリング平均（同じ窓幅で他の統計も可） ---
    # for win in rolling_windows:
    #     all_df[f'{col}_rolling_mean{win}'] = (
    #         all_df[col]
    #         .shift(1)                          # 現在行を除外したい場合
    #         .rolling(window=win, min_periods=1)
    #         .mean()
    #     )

    # --- expanding 平均 / 標準偏差 / z-score ---
    #   ・shift(1) を噛ませれば「t 時点の値を含まない」純粋な過去統計
    past_series = all_df[col].shift(1)

    all_df[f'{col}_exp_mean_to_t'] = past_series.expanding().mean()
    all_df[f'{col}_exp_std_to_t']  = past_series.expanding().std(ddof=0)

    # 0 除算を避けるため std==0 → NaN
    all_df.loc[all_df[f'{col}_exp_std_to_t'] == 0, f'{col}_exp_std_to_t'] = np.nan

    all_df[f'{col}_scaled'] = (
        (all_df[col] - all_df[f'{col}_exp_mean_to_t']) /
        all_df[f'{col}_exp_std_to_t']
    )

# ---- 計算後に再びtrain/testに分割 ----
train_df = (
    all_df[all_df['is_train']]
    .drop(columns=['is_train'])
    .copy()
)

test_df = (
    all_df[~all_df['is_train']]
    .drop(columns=['is_train', _common.TARGET_COL])  # 目的変数は残さない
    .copy()
)


### 気象データ

#### 湿度を露点温度へ変換

In [299]:
def calc_dew_point(temp_k, rh):
    """
    ケルビン温度と相対湿度から露点温度(ケルビン変換)を計算
    """
    import numpy as np
    temp_c = temp_k - 273.15  # K → ℃
    a = 17.62
    b = 243.12
    gamma = np.log(rh / 100) + (a * temp_c) / (b + temp_c)
    return (b * gamma) / (a - gamma) + 273.15

In [300]:
cities = ['madrid', 'barcelona', 'bilbao', 'seville', 'valencia']

# 各都市について露点温度を計算して新しい列に追加
for city in cities:
    temp_col = f'{city}_temp'
    hum_col = f'{city}_humidity'
    dew_col = f'{city}_dew_point'
    if temp_col in train_df.columns and hum_col in train_df.columns:
        train_df[dew_col] = calc_dew_point(train_df[temp_col], train_df[hum_col])
        train_df = train_df.drop(columns=hum_col)
    if temp_col in test_df.columns and hum_col in test_df.columns:
        test_df[dew_col] = calc_dew_point(test_df[temp_col], test_df[hum_col])
        test_df = test_df.drop(columns=hum_col)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


#### 人口比率による加重平均

|年|マドリード|バルセロナ|ビルバオ|セビリア|バレンシア|
|-|-|-|-|-|-|
|2015|約13.4%|約11.4%|約0.76%|約1.5%|約1.8%|
|2016|約13.6%|約11.5%|約0.76%|約1.5%|約1.8%|
|2017|約13.7%|約11.6%|約0.76%|約1.5%|約1.8%|
|2018|約13.9%|約11.7%|約0.75%|約1.5%|約1.8%|
|平均|13.7%|11.4%|0.76%|1.5%|1.8%|

In [301]:
pop_ratio = {
    'madrid': 0.137,
    'barcelona': 0.114,
    'bilbao': 0.0076,
    'seville': 0.015,
    'valencia': 0.018
}

In [302]:
weather_cols = ['wind_speed', 'temp', 'pressure', 'dew_point', 'clouds_all']

def add_weighted_weather(df, pop_ratio, weather_cols):
    cities = list(pop_ratio.keys())
    for w_col in weather_cols:
        weighted_vals = np.zeros(len(df))
        weight_sum = sum(pop_ratio[city] for city in cities)
        for city in cities:
            colname = f"{city}_{w_col}"
            weighted_vals += df[colname].fillna(0) * pop_ratio[city]
        df[f'weighted_{w_col}'] = weighted_vals / weight_sum
    # 元カラム名リストを返す（削除用）
    drop_cols = [f"{city}_{w_col}" for city in cities for w_col in weather_cols]
    return df.drop(columns=drop_cols)

# 実行例
train_df = add_weighted_weather(train_df, pop_ratio, weather_cols)
test_df = add_weighted_weather(test_df, pop_ratio, weather_cols)

#### 異常気象フラグ
高温・低温で一定水準を超えたらフラグを立てる
冷暖房度日（HDD/CDD）を調べたい

## 価格を特徴量に入れる

In [303]:
# #### ターゲットエンコーディング（曜日×時間）
# for df in [train_df, test_df]:
#     df['tmp_month'] = pd.to_datetime(df[_common.UNIQUE_KEY_COLS]).dt.month
#     df['tmp_weekday'] = pd.to_datetime(df[_common.UNIQUE_KEY_COLS]).dt.weekday
#     df['tmp_hour'] = pd.to_datetime(df[_common.UNIQUE_KEY_COLS]).dt.hour

# # === ターゲットエンコーディング（曜日 × 時間） ===
# group_cols = ['tmp_month', 'tmp_weekday', 'tmp_hour']
# encoded_col = "encoded_price_weekday_hour"

# # train から平均価格テーブルを作成
# mean_price_table = train_df.groupby(group_cols)[_common.TARGET_COL].mean().reset_index()
# mean_price_table = mean_price_table.rename(columns={_common.TARGET_COL: encoded_col})

# # train/test にエンコード値を結合
# train_df = train_df.merge(mean_price_table, on=group_cols, how='left')
# test_df = test_df.merge(mean_price_table, on=group_cols, how='left')

# # === エンコードに使った列は削除（sin/cosで代替済みのため） ===
# train_df = train_df.drop(columns=group_cols)
# test_df = test_df.drop(columns=group_cols)

## エンコーディング

In [304]:
def weather_group(col):
    col = str(col)
    if col.find('clear') != -1:
        return 1
    elif col.find('clouds') != -1:
        return 2
    else:
        return 3

In [305]:
# for col in _common.CATEGORY_COLS:
#     new_col = col + '_LabelEn'
#     train_df[new_col] = train_df[col].apply(weather_group)
#     train_df = train_df.drop(columns=col)
#     test_df[new_col] = test_df[col].apply(weather_group)
#     test_df = test_df.drop(columns=col)

#### 祝日フラグ
ChatGPTに2015年〜2018年の休日・祝日にフラグを立てる

In [306]:
from datetime import date
import holidays

es_holidays = holidays.ES()  # this is a dict-like object

In [ ]:
# train_df['is_holiday'] = (train_df[_common.UNIQUE_KEY_COLS].dt.date.isin(es_holidays).astype(int))
# test_df['is_holiday'] = (test_df[_common.UNIQUE_KEY_COLS].dt.date.isin(es_holidays).astype(int))

## 特徴量選択

In [308]:
train_df = train_df.dropna(axis=0, how='any')
data_x = train_df.drop(columns=[_common.TARGET_COL, _common.UNIQUE_KEY_COLS])
data_y = train_df[_common.TARGET_COL]

In [309]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(
    n_estimators=500,
    n_jobs=-1,
    max_depth=10,
    random_state=1234 
)
rf_reg = rf_reg.fit(data_x, data_y.values.ravel())

# 特徴量重要度の取得
fti = rf_reg.feature_importances_
feature_importance_df = pd.DataFrame({
    'feature': data_x.columns,
    'importance': fti
}).sort_values('importance', ascending=False).reset_index(drop=True)

feature_importance_df

,feature,importance
0,generation_fossil_hard_coal,0.198774
1,total_load_actual_exp_std_to_t,0.139592
2,gene_sum_exp_std_to_t,0.101518
3,dayofyear,0.088655
4,gene_sum_exp_mean_to_t,0.069087
5,weekday,0.050801
6,total_load_actual_scaled,0.048181
7,gene_sum_scaled,0.033086
8,weighted_wind_speed,0.032847
9,month,0.024911


## 前処理済みのファイル出力

In [310]:
train_df = train_df.dropna()

In [311]:
train_df.to_csv('../output/中間データ/学習用データ/train_preprocessed.csv', index=False)
test_df.to_csv('../output/中間データ/評価用データ/test_preprocessed.csv', index=False)

In [312]:
# カラム名のセットを取得
train_cols = set(train_df.columns)
test_cols = set(test_df.columns)

# どちらかにしかないカラム
only_in_train = train_cols - test_cols
only_in_test = test_cols - train_cols

print("train_dfのみに存在:", only_in_train)
print("test_dfのみに存在:", only_in_test)

train_dfのみに存在: {'price_actual'}
test_dfのみに存在: set()
